In [8]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [9]:
from proj1_helpers import *
import datetime
DATA_TRAIN_PATH = 'C:/Users/saadc/Desktop/EPFL/ML/ML_course/projects/project1/scripts/data/train.csv' # TODO: download train data and supply path here 
labels, input_data, ids = load_csv_data(DATA_TRAIN_PATH)

In [10]:
# standardize the data : 
x, mean_x, std_x = standardize(input_data)
y, tx = build_model_data(x, labels)

In [11]:
y.shape, tx.shape

((250000,), (250000, 31))

## Do your thing crazy machine learning thing here :) ...

# Least squares GD :

In [46]:
# from gradient_descent import *
from plots import gradient_descent_visualization
from gradient_descent import gradient_descent
# Define the parameters of the algorithm.
max_iters = 100
gamma = 0.01
initial_w = np.zeros((31))

w,loss = least_squares_GD(y, tx, initial_w, max_iters, gamma)

print("weights: ",w)
print("loss: ",loss)




weights:  [-0.03381892  0.11391166 -0.03929939 -0.02139935 -0.00740797  0.00642086
  0.05901993  0.0056357  -0.01375136 -0.01815117 -0.02155208 -0.01414564
 -0.01322023  0.00626779 -0.00716966 -0.01366855 -0.01368106 -0.02047625
 -0.01366609 -0.01365018 -0.01812219 -0.01363899 -0.02746221 -0.01383892
  0.02178121  0.02054982  0.02054818  0.00238098  0.00622045  0.00621068
 -0.02123287]
loss:  0.4186061051583552


# Least squares SGD : 

# Least squares Normal equations: 

# Ridge regression : 

# logistic regression : 

# Reg logistic regression : 


## Generate predictions and save ouput in csv format for submission:

In [ ]:
DATA_TEST_PATH = '' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)